<a href="https://colab.research.google.com/github/m-ni4da/main/blob/main/pyspark_%E5%95%8F%E9%A1%8C%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install findspark
!pip install pyarrow
!pip install xlrd
!pip install openpyxl
!pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=108818081fd853f757e2a03bf43b0f889517f562565d84a277a9a8a2140c4484
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructType, StructField
import pyspark.sql.functions as fn
from pyspark.sql.functions import col, lit
import pandas as pd

In [ ]:
spark = SparkSession.builder.appName('pyspark').getOrCreate()

In [ ]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:

        result = chardet.detect(file.read(10000))
        return result['encoding']

file_name = 'order_history.csv'  # CSVファイル名(別の階層にある場合はパス)
encoding = detect_encoding(file_name)
print(f"Detected encoding: {encoding}")

Detected encoding: ascii


In [ ]:
member_sdf = spark.read.format("csv").option("header", "true").option("encoding", "ascii").load("members.csv")
product_sdf = spark.read.format("csv").option("header", "true").option("encoding", "utf-8").load("products.csv")
order_sdf = spark.read.format("csv").option("header", "true").option("encoding", "ascii").load("order_history.csv")

In [ ]:
member_sdf.show()

+----------+---+------+
| member_id|age|gender|
+----------+---+------+
| cograph_1| 58|  Male|
| cograph_2| 44|Female|
| cograph_3| 50|Female|
| cograph_4| 62|Female|
| cograph_5| 59|  Male|
| cograph_6| 30|  Male|
| cograph_7| 50|Female|
| cograph_8| 38|  Male|
| cograph_9| 39|  Male|
|cograph_10| 44|Female|
|cograph_11| 41|  Male|
|cograph_12| 55|  Male|
|cograph_13| 48|  Male|
|cograph_14| 41|  Male|
|cograph_15| 44|Female|
|cograph_16| 43|Female|
|cograph_17| 55|Female|
|cograph_18| 38|Female|
|cograph_19| 43|Female|
|cograph_20| 31|  Male|
+----------+---+------+
only showing top 20 rows



In [ ]:
member_sdf.select("member_id").show()

+----------+
| member_id|
+----------+
| cograph_1|
| cograph_2|
| cograph_3|
| cograph_4|
| cograph_5|
| cograph_6|
| cograph_7|
| cograph_8|
| cograph_9|
|cograph_10|
|cograph_11|
|cograph_12|
|cograph_13|
|cograph_14|
|cograph_15|
|cograph_16|
|cograph_17|
|cograph_18|
|cograph_19|
|cograph_20|
+----------+
only showing top 20 rows



In [ ]:
member_sdf.select("gender", "member_id").show(5)

+------+---------+
|gender|member_id|
+------+---------+
|  Male|cograph_1|
|Female|cograph_2|
|Female|cograph_3|
|Female|cograph_4|
|  Male|cograph_5|
+------+---------+
only showing top 5 rows



In [ ]:
product_sdf.filter(col("price").cast("int") >= 1000).show()

+----------+------------------------+------+
|product_id|            product_name| price|
+----------+------------------------+------+
|         1|      ハンバーグステーキ|2200.0|
|         2|          シーザーサラダ|2100.0|
|         3|        クラムチャウダー|1400.0|
|         4|        マルゲリータピザ|1300.0|
|         5|スパゲッティカルボナーラ|1300.0|
|         6|          ローストチキン|1300.0|
|         7|          ビーフシチュー|1500.0|
|         8|              オムライス|2600.0|
|         9|      シーフードパエリア|2000.0|
|        10|            ラタトゥイユ|1900.0|
+----------+------------------------+------+



In [ ]:
member_sdf.filter(
    (col("age").cast("int") >= 20) & (col("gender") == "Female")
).show()

+----------+---+------+
| member_id|age|gender|
+----------+---+------+
| cograph_2| 44|Female|
| cograph_3| 50|Female|
| cograph_4| 62|Female|
| cograph_7| 50|Female|
|cograph_10| 44|Female|
|cograph_15| 44|Female|
|cograph_16| 43|Female|
|cograph_17| 55|Female|
|cograph_18| 38|Female|
|cograph_19| 43|Female|
|cograph_22| 47|Female|
|cograph_24| 33|Female|
|cograph_25| 63|Female|
|cograph_26| 25|Female|
|cograph_27| 40|Female|
|cograph_29| 55|Female|
|cograph_30| 55|Female|
|cograph_31| 42|Female|
|cograph_33| 31|Female|
|cograph_35| 37|Female|
+----------+---+------+
only showing top 20 rows



In [ ]:
product_sdf.filter(col("price") <=800).agg(fn.avg("price")).show()

+----------+
|avg(price)|
+----------+
|     574.2|
+----------+



In [ ]:
member_sdf.groupBy("gender").agg(fn.count("member_id")).show()

+------+----------------+
|gender|count(member_id)|
+------+----------------+
|Female|              80|
|  Male|             120|
+------+----------------+



In [ ]:
order_sdf.groupBy("product_id").agg(
    fn.sum(col("quantity").cast("int")).alias("cnt"),
    ).select("product_id", "cnt").orderBy(col("product_id").cast("int")).show()

+----------+----+
|product_id| cnt|
+----------+----+
|         1| 345|
|         2| 329|
|         3| 334|
|         4| 301|
|         5| 309|
|         6| 154|
|         7| 149|
|         8| 166|
|         9| 182|
|        10| 185|
|        11| 552|
|        12| 534|
|        13| 586|
|        14|1260|
|        15| 505|
+----------+----+

